# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

### 项目难点和解决办法：  
1. iex cloud不再提供sandbox，需要自己注册一个试用账号获取token  
2. iex cloud的url地址更换了，根据documentation找到新的api_url  
3. 使用pd.DataFrame().append()将series添加到df的时候会警告append的功能即将被取消，建议使用pd.concat()  
4. pd.series是一个列，无法直接添加到df，需要通过serie.to_frame().T转换成行  
5. batch api call的url需要更换，call出来的数据是一个列表中包含所有股票的数据，每一个股票的数据是一个字典的数据结构，需要通过for loop获取每一个股票的数据（字典）再读取数据


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
print(type(stocks))

<class 'pandas.core.frame.DataFrame'>


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from verysecrets import IEX_CLOUD_API_TOKEN

https://stackoverflow.com/questions/73055157/what-does-importerror-cannot-import-name-randbits-mean
Somehow anaconda3/Lib/secrets.py got overwritten. Numpy relies on files in this directory called random.py and secrets.py so if you have files with those names numpy will not load.  
solution:  
renamed the secrets.py to verysecrets.py

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [138]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()[0]
# print(data.status_code) check whether the connet works, 200 means work well
print(data)

{'avgTotalVolume': 56318533, 'calculationPrice': 'close', 'change': -0.92, 'changePercent': -0.00518, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': None, 'highTime': None, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 176.21, 'iexCloseTime': 1699278048944, 'iexLastUpdated': 1699278048944, 'iexMarketPercent': 0.000299469278018955, 'iexOpen': 176.21, 'iexOpenTime': 1699278048944, 'iexRealtimePrice': 176.21, 'iexRealtimeSize': 50, 'iexVolume': 806, 'lastTradeTime': 1699041613847, 'latestPrice': 176.65, 'latestSource': 'Close', 'latestTime': 'November 3, 2023', 'latestUpdate': 1699041600060, 'latestVolume': None, 'low': None, 'lowSource': None, 'lowTime': None, 'marketCap': 2761787082800, 'oddLotDelayedPrice': None,

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [140]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [168]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
initial_dataframe = pd.DataFrame(columns = my_columns)

In [190]:
test_serie = pd.Series([symbol, price, market_cap, 'N/A'], index=my_columns)

In [191]:
final_dataframe = pd.concat([initial_dataframe, test_serie.to_frame().T], ignore_index=True)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,176.65,2761787082800,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [196]:
# final_dataframe = pd.DataFrame(columns = my_columns)
# for symbol in stocks['Ticker']:
#     api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
#     data = requests.get(api_url).json()
#     final_dataframe = final_dataframe.append(
#                                         pd.Series([symbol, 
#                                                    data['latestPrice'], 
#                                                    data['marketCap'], 
#                                                    'N/A'], 
#                                                   index = my_columns), 
#                                         ignore_index = True)

In [ ]:
# final_dataframe

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [9]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i: i+n]
        
# Yield is a statement that is used in functions and that function can provide a result back to the caller 
# (like a return in a normal function), 
# but the function will then continue to execute with all of the local variables staying intact.

In [10]:
symbol_group = list(chunks(stocks['Ticker'], 100))
symbol_group[0:2]

[0         A
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
       ...  
 95     CINF
 96       CL
 97      CLX
 98      CMA
 99    CMCSA
 Name: Ticker, Length: 100, dtype: object,
 100     CME
 101     CMG
 102     CMI
 103     CMS
 104     CNC
        ... 
 195    FTNT
 196     FTV
 197      GD
 198      GE
 199    GILD
 Name: Ticker, Length: 100, dtype: object]

In [34]:
symbol_strings = []
for i in range(0, len(symbol_group)):
    symbol_strings.append(','.join(symbol_group[i]))
#     print(symbol_strings[i])
symbol_strings

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

In [192]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for stock in data:
        series_to_add = pd.Series([stock['symbol'], stock['latestPrice'], stock['marketCap'], 'N/A'], index=my_columns)
        final_dataframe = pd.concat([final_dataframe, series_to_add.to_frame().T], ignore_index=True)

In [193]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,107.53,31461909466,N/A
1,AAL,11.66,7620282813,N/A
2,AAP,56.66,3369584988,N/A
3,AAPL,179.23,2802123401360,N/A
4,ABBV,141.2,249224591216,N/A
...,...,...,...,...
500,YUM,126.39,35415903806,N/A
501,ZBH,109.3,22839779846,N/A
502,ZBRA,206.02,10581105616,N/A
503,ZION,34.18,5063600646,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [201]:
portfolio_size = input('Enter value of your portfolio: ')

try:
    val = float(portfolio_size)
except ValueError:
    print('Integer is required')
    portfolio_size = input('Enter value of your portfolio: ')
    val = float(portfolio_size)

Enter value of your portfolio: 100000000


In [202]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
    
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,107.53,31461909466,1841
1,AAL,11.66,7620282813,16982
2,AAP,56.66,3369584988,3494
3,AAPL,179.23,2802123401360,1104
4,ABBV,141.2,249224591216,1402
...,...,...,...,...
500,YUM,126.39,35415903806,1566
501,ZBH,109.3,22839779846,1811
502,ZBRA,206.02,10581105616,961
503,ZION,34.18,5063600646,5793


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [241]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [242]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

doller_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border':1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border':1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [243]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)

# writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [244]:
column_format = {
    'A': ['Tricker', string_format],
    'B': ['Stock Price', doller_format],
    'C': ['Makret Capitalization', doller_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_format.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_format[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_format[column][0], column_format[column][1] )


## Saving Our Excel Output

Saving our Excel file is very easy:

In [247]:
writer.save()

/var/folders/1m/d9zdkrs55x72dhfr9qb59_700000gn/T/ipykernel_2772/934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
